importing packages

In [53]:
!pip install --upgrade langchain_core langchain_chroma langchain_text_splitters langchain langchain_google_genai sentence_transformers langsmith gradio gdown

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [54]:
import pandas as pd
import numpy as np
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.documents import Document
from kaggle_secrets import UserSecretsClient
import gdown
from langchain.load import dumps, loads
import gradio as gr

In [3]:
user_secrets = UserSecretsClient()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = user_secrets.get_secret("LANGCHAIN_KEY")

In [4]:
data_directory='/kaggle/input/sanad-dataset'
data=[]
for folder in os.listdir(data_directory):
    for file in os.listdir(f'{data_directory}/{folder}'):

            file = open(f"{data_directory}/{folder}/{file}", "r",encoding="utf8")
            content = file.read()

            data.append({'category': folder, 'Content': content})

            file.close()

In [5]:
data[0]

{'category': 'Finance',
 'Content': 'دبي - "الخليج":حصدت شعاع كابيتال جائزة "أفضل شركة لإدارة الأصول في الإمارات" خلال حفل توزيع جوائز الأعمال المصرفية في الشرق الأوسط لعام 2014 الذي تنظمه مجلة إيميا فايننس . وتم اختيار شعاع لنيل الجائزة للسنة الخامسة على التوالي، تقديراً للأداء القوي الذي يحققه فريقها لإدارة الأصول .وقال الشيخ مكتوم بن حشر آل مكتوم، رئيس مجلس الإدارة التنفيذي لشعاع كابيتال: "نحن فخورون بالتكريم الذي حصل عليه فريقنا لإدارة الأصول مرّة جديدة، وذلك تقديراً لتميّزه في تحقيق مستوى متسق من العائدات لصالح العملاء . ويعدّ الفوز بهذه الجائزة لخمس مرّات متتالية إنجازاً كبيراً، ودافعاً قوياً لفريق إدارة الأصول ليواصلوا تحقيق هذا الأداء المتميّز" .وبلغ العائد الذي حققه الصندوقان الرئيسيان لدى شعاع لإدارة الأصول، وهما صندوق البوابة العربية وصندوق بوابة الإمارات، 2 .16% و2 .30% على التوالي خلال الفترة الماضية من العام 2014 . وتمتلك صناديق شعاع، التي أطلقت في العام ،1999 أطول سجل استثماري في المنطقة وقد حصل فريق إدارة الأصول لدى الشركة على مجموعة كبيرة من الجوائز خلال السنوات الأخير

In [6]:
documents = [Document(page_content=text["Content"]) for text in data[:40000]]

splitting data into documents

In [7]:
len(documents)

40000

Embedding to find similarity between texts and find relevant

In [8]:
class embedding:
    def __init__(self):
        self.model = SentenceTransformer('all-mpnet-base-v2')
    def embed_documents(self,docs):
        embeddings= self.model.encode(docs)
        return embeddings.tolist()
    def embed_query(self,query):
        return self.model.encode(query).tolist()

In [9]:
embed_model=embedding()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
result=embed_model.embed_query("ما الجائزة التي حصدتها شعاع كابيتال ؟")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
len(result)

768

In [12]:
directory = "/kaggle/working/"

In [ ]:
url = "https://drive.google.com/drive/u/0/folders/149jD_H7oQ3JEp6D1UBaMTNIyMOs8yz06"
gdown.download_folder(url)

In [59]:
try:
    vector_database = Chroma(persist_directory="/kaggle/working/vector", embedding_function=embed_model)
except Exception as e:
    # If it fails, initialize with documents
    vector_database = Chroma.from_documents(documents, embedding=embed_model, persist_directory = directory)

In [61]:
retriever=vector_database.as_retriever(search_type="similarity",search_kwargs={'k':3})

construct template

In [62]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
google_api_key = user_secrets.get_secret("GOOGLE_AI_STUDIO2")
llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=google_api_key,temperature=0)

In [63]:

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five arabic
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines without any additional thoughts outside the answers. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)


In [64]:


def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
retrieval_chain = generate_queries | retriever.map() | get_unique_union




In [65]:


# RAG
# template = """Answer the following question based on this context:

# {context}

# Question: {question}
# """

template="""
you are an AI powered QA Assistant to provide accurate, contextually relevant answers to customer questions.
at the end of the answer you have to thank the user.
the answer in arabic and in details.
you should answer based on the context provided:
{context}

what to do if answer is not included in the prompt for context:

1.you should appoligize to the user and say that you dont have the answer in your informations

for answer:
1. the output must be in details based on the context provided

Question: {question}

answer:
"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

final_rag_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retrieval_chain, "question": RunnablePassthrough()}
).assign(answer=final_rag_chain)

# final_rag_chain.invoke({"question":question})



In [66]:
import gradio as gr

def getting_answers(question, history=[]):
    rag_answer = rag_chain_with_source.invoke(question)
    answer = rag_answer["answer"] + "\nبناءا على اقرب معلومات لدي : \n" + "\n".join([docs.page_content for docs in rag_answer["context"]])
    history.append((question, answer))

    return history,history


demo = gr.Interface(
    fn=getting_answers,
    inputs=["text", "state"],  # "text" for user input, "state" to keep track of the chat history
    outputs=["chatbot", "state"],  # "chatbot" for chat UI, "state" to store chat history
)

demo.launch()

Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://704700fa5bdaa05f76.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
